In [9]:
#Bouaoune Youghourta BOUY28059509 
import pandas as pd
import numpy as np
from scipy import linalg
from scipy.spatial import distance
import heapq
from scipy.spatial import Delaunay
# from google.colab import files
# data = files.upload()

df = pd.read_csv('notes.csv')
dfF = pd.read_csv('films.csv', encoding="latin1")
df.columns
df

,IdUtilisateur,IdFilm,Note,Horodatage
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [10]:
#répnce 1):Quel est le nombre de notes donn´ees par tous les utilisateurs ?
# le nombre de notes :
df_note = df['Note']
df_note.notnull().count()


100836

In [4]:
#réponce 2):Quel est le nombre d’utilisateurs ?
# le nombre d'utlisateur:
df_utilisateur = df['IdUtilisateur']
df_utilisateur.nunique()

610

In [6]:
#2.  Quel est le nombre de films ?
#le numero de films
df_film = dfF['IdFilm']
df_film.notnull().count()

9742

In [8]:
#réponce 3):3. Quel est le nombre moyen de notes par utilisateur ?
# le nombre moyenne de note par utilisateur

df.groupby(['IdUtilisateur'])['Note'].agg(['mean'])

,mean
IdUtilisateur,
1,4.366379
2,3.948276
3,2.435897
4,3.555556
5,3.636364
...,...
606,3.657399
607,3.786096
608,3.134176


In [8]:
#réponce 4):Quel est le nombre moyen de notes par film ?
# le nombre moyenne de note par filme
df_n_m_filme = df.groupby(['IdFilm'])['Note'].agg(['mean'])
df_n_m_filme

,mean
IdFilm,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429
...,...
193581,4.000000
193583,3.500000
193585,3.500000


In [9]:
#réponce 5):Quel film a la note moyenne la plus basse?
# le filme qui a la petite moyenne

Min_film = df_n_m_filme.loc[(df_n_m_filme['mean'] == min(df_n_m_filme['mean']))]
print(Min_film)
Min_film = dfF.loc[(dfF['IdFilm'] == 3604 )]
print(Min_film)
#réponce 6):

        mean
IdFilm      
3604     0.5
3933     0.5
4051     0.5
4371     0.5
4580     0.5
...      ...
165645   0.5
167296   0.5
173307   0.5
175475   0.5
184641   0.5

[96 rows x 1 columns]
      IdFilm         Titre    Genres
2689    3604  Gypsy (1962)  Musicale


In [10]:
#réponce 6):Quel film a la note moyenne la plus ´elev´ee ?
#print(max(df_n_m_filme['mean']))

Max_film = df_n_m_filme.loc[(df_n_m_filme['mean'] == max(df_n_m_filme['mean']))]
print(Max_film)
Min_film = dfF.loc[(dfF['IdFilm'] == 53 )]
print(Min_film)

        mean
IdFilm      
53       5.0
99       5.0
148      5.0
467      5.0
495      5.0
...      ...
176601   5.0
179133   5.0
179135   5.0
184245   5.0
187717   5.0

[296 rows x 1 columns]
    IdFilm            Titre          Genres
48      53  Lamerica (1994)  Aventure|Drame


In [11]:
#reponce 1:Filtrez les utilisateurs qui ont vot´es moins de 100 films et les films qui
#sont vot´es par moins de 10 utilisateurs? Remarque: vous pouvez utiliser
#pandas.DataFrame.groupby(). Dans la suite, on utilisera uniquement la
#matrice filtr´ee.
nb_100 = df.groupby(['IdUtilisateur'])['Note'].size().sort_values(ascending=False)
nb_100 = nb_100.loc[(nb_100 < 100)]
print(nb_100)

nb_10 = df.groupby(['IdFilm'])['Note'].size().sort_values(ascending=False)
nb_10 = nb_10.loc[(nb_10 < 10)]

print(nb_10)

IdUtilisateur
323    98
16     98
215    98
159    97
93     97
       ..
442    20
569    20
320    20
576    20
53     20
Name: Note, Length: 362, dtype: int64
IdFilm
674       9
688       9
95105     9
2320      9
678       9
         ..
4093      1
4089      1
58351     1
4083      1
193609    1
Name: Note, Length: 7455, dtype: int64


In [12]:
#réponce 2):Construisez la matrice utilisateur-film? Remarque: vous pouvez utiliser
#la m´ethode pandas.DataFrame.pivot().
d = pd.pivot_table(df,index='IdUtilisateur',columns='IdFilm',values='Note')
d.fillna(0,inplace=True)
index = d.index.union(d.columns)
d

IdFilm,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
IdUtilisateur,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#réponce 3):Calculez le taux de parcimonie de la matrice utilisateur-film en utilisant la
#formule 1?
d = pd.pivot_table(df,index='IdUtilisateur',columns='IdFilm',values='Note')
index = d.index.union(d.columns)
A = np.count_nonzero(~np.isnan(d))
B = d.size
s = A/B
s

0.016999683055613623

In [14]:
################################5.4########################################
#1)Appliquez SVD sur la matrice utilisateur-film pour r´eduire la dimension de
#cette matrice `a une dimension plus faible associ´ee aux ‘goˆuts des utilisateurs
#/ genres de films” les plus pertinents? Remarque: vous devez conserver
#les k valeurs singuli`eres les plus grandes avec k entre 5 et 20.
df_notes = pd.read_csv('notes.csv')
df_films = pd.read_csv('films.csv', encoding="latin1")
M = pd.pivot_table(df,index='IdUtilisateur',columns='IdFilm',values='Note')
M.fillna(0,inplace=True)

U, s, Vh = linalg.svd(M, check_finite=False, full_matrices = False)

## reduce dimension
k = 20
Vh = Vh[:k] # keep 20 first lines
s = s[:k] # keep 20 first values
U = U[:,:k] # keep 20 first collumns
#s = np.diag(s)
Vh = Vh.T



In [15]:
#2)Cherchez les 5 films les plus similaires au premier film ? Remarque: vous
#devez utiliser la fonction scipy.spatial.distance.cosine sur les lignes de la
#matrice V (apr`es la r´eduction de la dimension).

n=5
heap = [(-distance.cosine(Vh[0], Vh[i]),i) for i in range(1,n+1)]
heapq.heapify(heap) # using a heap is more efficient than sorting
for i in range(n+1, Vh.shape[0]):
    # using negative distance because pop removes the smallest value
    heapq.heappushpop(heap, (-distance.cosine(Vh[0], Vh[i]),i))
n_smallest = [heapq.heappop(heap) for i in range(n)]
dfF.iloc[[id for _,id in n_smallest[::-1]] ]


,IdFilm,Titre,Genres
506,588,Aladdin (1992),Aventure|Animation|Enfants|Comédie|Musicale
504,586,Home Alone (1990),Enfants|Comédie
815,1073,Willy Wonka & the Chocolate Factory (1971),Enfants|Comédie|Fantaisie|Musicale
963,1264,Diva (1981),Action|Drame|Mystère|Romance|Thriller
898,1196,Star Guerres: Episode V - The Empire Strikes B...,Action|Aventure|Sci-Fi


In [16]:
#3)Affichez les films recommend´es pour le premier utilisateur? Remarque:
#vous devez afficher l’identifiant de chaque film recommand´e, son titre et
#son genre. Il ne faut pas inclure les films que l’utilisateur a d´ej`a regard´es.
## n most recomeneded movies to the first user
n=10
heap = []
for i in range(Vh.shape[0]):
    if not M.values[0][i]: # only use unrated movies
        estimated_note = U[0].dot(s[0] * Vh[i])
        if len(heap)<n:
            heapq.heappush(heap, (estimated_note,i))
        else:
            heapq.heappushpop(heap, (estimated_note,i))
n_biggest = [heapq.heappop(heap) for i in range(n)]
dfF.iloc[[id for _,id in n_biggest[::-1]]]

,IdFilm,Titre,Genres
1444,1967,Labyrinth (1986),Aventure|Fantaisie|Musicale
901,1199,Brazil (1985),Fantaisie|Sci-Fi
957,1258,"Shining, The (1980)",Horreur
507,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
1056,1373,Star Trek V: The Final Frontier (1989),Action|Sci-Fi
2109,2803,"Pelican Brief, The (1993)",Crime|Drame|Mystère|Romance|Thriller
2077,2761,"Iron Giant, The (1999)",Aventure|Animation|Enfants|Drame|Sci-Fi
2194,2917,Body Heat (1981),Crime|Thriller
1066,1385,Under Siege (1992),Action|Drame|Thriller
1186,1583,"Simple Wish, A (1997)",Enfants|Fantaisie
